In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelBinarizer
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_curve, auc

from imblearn.over_sampling import SMOTE  # imblearn yüklü değilse: pip install imblearn

# Veri yükleme
data = pd.read_excel('paper.xlsx')

# Veri genel bilgisi ve önizleme
print(data.info())
print(data.head())

# Eksik veri kontrolü
print("Eksik veriler:\n", data.isnull().sum())

# GPA'yı sınıflara ayırma
def gpa_to_label(gpa):
    if gpa >= 3.0:
        return 'High'
    elif gpa >= 2.0:
        return 'Medium'
    else:
        return 'Low'

data['GPA_Class'] = data['GPA'].apply(gpa_to_label)

# Eksik verileri temizleme
data = data.dropna()

# Sayısal sütunlar
numeric_columns = ['Time_Friends', 'Time_SocicalMedia', 'Time_Studying']
numeric_columns = [col for col in numeric_columns if col in data.columns]

# Sayısal sütunları ölçeklendirme
scaler = StandardScaler()
data[numeric_columns] = scaler.fit_transform(data[numeric_columns])

# Hedef değişken ve özellikler
y = data['GPA_Class']
X = data.drop(['GPA', 'GPA_Class'], axis=1)

# Korelasyon matrisi görselleştirme
numeric_data = data.select_dtypes(include=[np.number])

plt.figure(figsize=(10,8))
sns.heatmap(numeric_data.corr(), annot=True, cmap='coolwarm', fmt=".2f")
plt.title("Korelasyon Matrisi")
plt.show()

# Orijinal sınıf dağılımı
print("Orijinal sınıf dağılımı:\n", y.value_counts())

# SMOTE ile oversampling
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

print("Oversampling sonrası sınıf dağılımı:\n", pd.Series(y_resampled).value_counts())

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

# Naive Bayes Model Eğitimi
model = GaussianNB()
model.fit(X_train, y_train)

# Tahminler
y_pred = model.predict(X_test)

# --- ROC Eğrisi için hazırlık ---

# Hedef değişkeni one-hot encoding yapalım
lb = LabelBinarizer()
y_test_bin = lb.fit_transform(y_test)

# Eğer sadece iki sınıf varsa, y_test_bin 1 sütun olabilir, bunu 2 sütun yapalım
if y_test_bin.shape[1] == 1:
    y_test_bin = np.hstack((1 - y_test_bin, y_test_bin))

# Modelin her sınıf için olasılık tahminleri
y_pred_prob = model.predict_proba(X_test)

# Sınıf sayısı
n_classes = y_pred_prob.shape[1]

# ROC eğrisi çizimi
fpr = dict()
tpr = dict()
roc_auc = dict()

plt.figure(figsize=(8, 6))

colors = ['blue', 'orange', 'green']
for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(y_test_bin[:, i], y_pred_prob[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])
    plt.plot(fpr[i], tpr[i], color=colors[i], lw=2,
             label=f'{lb.classes_[i]} (AUC = {roc_auc[i]:.2f})')

plt.plot([0, 1], [0, 1], 'k--', lw=2)  # Rastgele sınıflandırıcı çizgisi
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Eğrisi (One-vs-Rest)')
plt.legend(loc="lower right")
plt.show()

# Performans sonuçları
print("Doğruluk:", accuracy_score(y_test, y_pred))
print("Sınıflandırma Raporu:\n", classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

# Naive Bayes modelinde özellik önemleri direkt yoktur,
# ancak yine de sayısal değişkenlerin GPA sınıfına göre dağılımını gösterelim

# Boxplot ile sayısal değişkenlerin GPA sınıflarına göre dağılımı
for col in numeric_columns:
    plt.figure(figsize=(8,6))
    sns.boxplot(x='GPA_Class', y=col, data=data)
    plt.title(f"{col} Dağılımı GPA Sınıflarına Göre")
    plt.show()
